In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
!nvidia-smi
!nvcc --version

# Test if your notebook is running on Google Colab
# You'll use this when choosing between doing interaction via ipywidgets or not.
try:
    import google.colab
    _ON_COLAB = True
except:
    _ON_COLAB = False

print('Running on Google Colab = ', _ON_COLAB)
import os
# Definir o caminho base
if _ON_COLAB:
    BASE_PATH = "/content"
else:
    BASE_PATH = "/lapix" # trocar para o base path do seu computador se estiver rodando localmente, deixar /lapix se estiver rodando nos conteiners lapix

os.chdir(BASE_PATH) # garantir que está executando no caminho base definido

Sun Dec  8 18:14:59 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          On  |   00000000:1B:00.0 Off |                    0 |
| N/A   28C    P0             68W /  700W |      24MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
INSTALL_LIBS = False # Setar para True para instalar localmente também (Não é necessário se estiver rodando nos conteiners Lapix)

if _ON_COLAB:
    INSTALL_LIBS = True

if INSTALL_LIBS:
    !pip install lapixdl
    !pip install -U albumentations
    !pip install fastai
    !pip install wandb -qU

In [5]:
!pip install loguru requests tqdm geopandas

DEPRECATION: Loading egg at /root/anaconda3/envs/lapixdl/lib/python3.11/site-packages/paddleseg-0.0.0.dev0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [6]:
from loguru import logger
import geopandas as gpd

class Utils:
    def __init__(self):
        pass


    def construct_query_for_sentinel2_products(self, footprint: str, start_date: str, end_date: str, cloud_cover_percentage: str, type,
              platform_name: str):
        """
        Create a query for downloading Sentinel data based on specified parameters.

        Parameters:
        ------------
        footprint: str
            The spatial geometry (POLYGON) of the area of interest.
        start_date: str
            The start date for the time range of interest in the format 'YYYY-MM-DD'.
        end_date: str
            The end date for the time range of interest in the format 'YYYY-MM-DD'.
        cloud_cover_percentage: str
            The maximum allowable cloud cover percentage.
        type: str or list
            Type of MSI to download
        platform_name: str, optional
            The name of the Sentinel platform (default: 'SENTINEL-2').

        Returns:
        params: str
            The query string for downloading Sentinel data.
        """

        logger.info("Creating query to download Sentinel data...")
        try:

            if footprint and start_date and end_date and platform_name and cloud_cover_percentage and type:
                footprint = footprint.replace(" ", "%20")
                if isinstance(type, str):
                    params = f"?&$filter=(Collection/Name%20eq%20%27{platform_name}%27%20and%20(Attributes/OData.CSC.StringAttribute/any(att:att/Name%20eq%20%27instrumentShortName%27%20and%20att/OData.CSC.StringAttribute/Value%20eq%20%27MSI%27)%20and%20Attributes/OData.CSC.DoubleAttribute/any(att:att/Name%20eq%20%27cloudCover%27%20and%20att/OData.CSC.DoubleAttribute/Value%20le%20{cloud_cover_percentage})%20and%20(contains(Name,%27{type}%27)%20and%20OData.CSC.Intersects(area=geography%27SRID=4326;{footprint}%27)))%20and%20Online%20eq%20true)%20and%20ContentDate/Start%20ge%20{start_date}T00:00:00.000Z%20and%20ContentDate/Start%20lt%20{end_date}T23:59:59.999Z&$orderby=ContentDate/Start%20desc&$expand=Attributes&$count=True&$top=50&$expand=Assets&$skip=0"
                    logger.success("Query created successfully")
                    return params
                elif isinstance(type, list):
                    params = f"?&$filter=(Collection/Name%20eq%20%27{platform_name}%27%20and%20(Attributes/OData.CSC.StringAttribute/any(att:att/Name%20eq%20%27instrumentShortName%27%20and%20att/OData.CSC.StringAttribute/Value%20eq%20%27MSI%27)%20and%20Attributes/OData.CSC.DoubleAttribute/any(att:att/Name%20eq%20%27cloudCover%27%20and%20att/OData.CSC.DoubleAttribute/Value%20le%20{cloud_cover_percentage})%20and%20((contains(Name,%27{type[0]}%27)%20and%20OData.CSC.Intersects(area=geography%27SRID=4326;{footprint}%27))%20or%20(contains(Name,%27{type[1]}%27)%20and%20OData.CSC.Intersects(area=geography%27SRID=4326;{footprint}%27))))%20and%20Online%20eq%20true)%20and%20ContentDate/Start%20ge%20{start_date}T00:00:00.000Z%20and%20ContentDate/Start%20lt%20{end_date}T23:59:59.999Z&$orderby=ContentDate/Start%20desc&$expand=Attributes&$count=True&$top=50&$expand=Assets&$skip=0"
                    logger.success("Query created successfully")
                    return params
                else:
                    raise ValueError("Please provide valid values for type.")
            else:
                raise ValueError("Please provide valid values for all required parameters.")
        except Exception as e:
            logger.error(f"An error occurred: {str(e)}")
            raise

    def retrieve_products_info(self, products):
        """
        Retrieve information about Sentinel products.

        Parameters:
        -----------
        products: list
            List of Sentinel products, each represented as a dictionary.

        Returns:
        ---------
        list:
            A list containing information about each product. Each element in the list is a sublist with the following format:
            [product_id, product_name, product_s3path, product_origin_date, product_tile, product_platform_name, product_type]
        """
        result_list = []
        logger.info(f"Retrieving information about {len(products)} products...")

        for i, product in enumerate(products):
            logger.info(f"Retrieving information about product {i+1}")
            product_id = product['Id']
            product_name = product['Name']
            product_s3path = product["S3Path"]
            product_origin_date = product['ContentDate']['End']

            # Creating output directory to save downloaded data
            product_tile = product_name.split("_")[5]
            product_platform_name = product_s3path.split('/')[2]
            product_type = product_name.split('_')[1][-3:]

            # Append product information to the result list
            result_list.append([product_id, product_name, product_s3path, product_origin_date, product_tile, product_platform_name, product_type])
            logger.success(f"ID, Name, S3Path, Origin Date, Tile, Plataform Name and Type successfully retrieved for product {i+1}")

        logger.success("Information successfully retrieved for all products. A list with these informations were created!")
        return result_list

    def modify_string(self, url):
        """
        Modify a URL by replacing the last occurrence of "Nodes" with "$value".

        Parameters:
        -----------
        url: str
            The input URL to be modified.

        Returns:
        ---------
        modified_url: str
            The modified URL with the last "Nodes" replaced by "$value".
        ValueError
            If "Nodes" is not found in the input URL.
    """
        last_nodes_index = url.rfind("Nodes")

        if last_nodes_index != -1:
        # Remove the last "Nodes" and replace it with "/$value"
            modified_url = url[:last_nodes_index] + "$value"
            return modified_url
        else:
            raise ValueError("Error: 'Nodes' not found in the URL")

    def shapefile_to_polygon(self, shp_path):
        """
        Transform a shapefile in a POLYGON geometry.

        Parameters:
        -----------
        shp_path: dtr
            Path to shapefile

        Returns:
        ---------
        pol: str
            The shapefile polygon has a string
        ValueError
            If shapefile is empty
        """
        gdf = gpd.read_file(shp_path)

        if gdf.empty:
          raise ValueError("Error: Shapefile is empty.")

        # Extract first geometry from GeoDataFrame
        pol = gdf.geometry.iloc[0]

        if not pol.is_valid or not pol.geom_type == 'Polygon':
          raise ValueError("Error: Shapefile's geometry is not valide or it is not a polygon.")


        return pol

In [7]:
from datetime import datetime, timedelta
from loguru import logger

class Dates:
    def __init__(self):
        pass

    def is_token_expired(self, dt_access_token, dt_download):
            """
            Check if the provided access token has expired.

            Parameters:
            -----------
            access_token: str
                The access token to be checked for expiration.

            Returns:
            --------
            bool
                True if the token has expired, False otherwise.
            """

            # Check if the current time is 9 minutes or more past the expiration time
            return dt_download > dt_access_token + timedelta(minutes=9)

In [8]:
import os
from loguru import logger
from datetime import datetime


class Files:
    def __init__(self):
        pass


    def save_path_sentinel2_data(self, dir_out, etc, date, type, tile, band=None):
        """
        Generate a directory path for Sentinel-2 data and create it if it doesn't exist.

        Parameters:
        ------------
        dir_out: str
            The base output directory.
        date: str
            The date in the format '%Y-%m-%dT%H:%M:%S.%fZ'.
        etc: str
            Additional information for the directory path.
        tile: str
            The tile information for the directory path.
        band: str, optional
            The band information for the directory path.

        Returns:
        ----------
        str
            The path of the generated or existing directory.
        """
        dt_object = datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.%fZ')
        # Format the datetime object as 'YYYY/MM/DD'
        date = dt_object.strftime('%Y/%m/%d')

        if band is not None:
            directory = os.path.join(dir_out, etc, date, type, tile, band)
        else:
            directory = os.path.join(dir_out, etc, date, type, tile)

        if os.path.exists(directory):
            logger.warning(f"Directory {directory} already exists.")
            return directory
        else:
            os.makedirs(directory, exist_ok=True)
            logger.success(f"Directory {directory} created!")
            return directory


    def create_output_folders(self, output_base_path, products_info, bands_dict):
        """
        Create, if necessary, folders to save the downloaded bands.

        Parameters:
        -----------
        output_base_path: str
            The base path where the output folders will be created.
        products_info: list
            A list of product information, where each item contains:
            [product_id, product_name, product_path, product_origin_date, product_tile, product_platform_name, product_type]
        bands_dict: dict
            A dictionary containing information about bands for different product types and resolutions.

        Returns:
        --------
        created_directories: list
            A list of paths to the created output directories.
        """
        logger.info("Creating, if necessary, folders to save the downloaded bands...")
        created_directories = []
        for product_info in products_info:
            product_type = product_info[6]
            product_tile = product_info[4]
            product_date = product_info[3]
            product_plataform = product_info[5]
            if product_type == "L1C":
                directory_path = self.save_path_sentinel2_data(output_base_path, product_plataform, product_date, product_tile, product_type)
                created_directories.append(directory_path)
            elif product_type == "L2A":
                for resolution, _ in bands_dict.items():
                    if resolution == "L1C":
                        pass
                    else:
                        l2a_dict = bands_dict.get("L2A", {})
                        for resolution_key in l2a_dict:
                            directory_path = self.save_path_sentinel2_data(output_base_path, product_plataform, product_date, product_tile, product_type, resolution_key)
                            created_directories.append(directory_path)

        return created_directories


    def check_file_exist(self, base_dir, etc, date, tile, type, filename, resolution = None):
        """
        Check if a file exists based on the specified parameters.

        Parameters:
        -----------
        base_dir: str
            The base directory where the file is expected to exist.
        etc: str
            Additional subdirectory for categorization (e.g., product platform or other metadata).
        date: str
            The date of the product in 'YYYY-MM-DDTHH:MM:SS.sssZ' format.
        tile: str
            The tile identifier associated with the product.
        type: str
            The type of the product (e.g., 'L1C', 'L2A').
        filename: str
            The name of the file to check.
        resolution: str, optional
            The resolution category (e.g., 'resolution10m', 'resolution20m').
            If provided, the file path will include this resolution subdirectory.

        Returns:
        --------
        exists: bool
            True if the file exists, False otherwise.
        file_path: str
            The full path to the checked file.

        """
        date_obj = datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.%fZ')
        # Format datetime object as 'YYYY/MM/DD'
        date = date_obj.strftime('%Y/%m/%d')

        if resolution is None:
            file = os.path.join(base_dir, etc, date, tile, type, filename)
            return os.path.exists(file), file
        else:
            file = os.path.join(base_dir, etc, date, tile, type, resolution, filename)
            return os.path.exists(file), file




In [9]:
import requests
from tqdm import tqdm
from loguru import logger
from datetime import datetime


class Connections:
    def __init__(self):
        pass


    def access_token(self, username: str, password: str) -> str:
        """
        Obtain an access token for accessing the Sentinel API.

        Parameters:
        -----------
        username: str
            The username for authentication.
        password: str
            The password for authentication.

        Returns:
        --------
        access_token, refresh_token, dt_now: Tuple[str, str, datetime]
            A tuple containing the access token, refresh token, and the current datetime when the tokens were obtained.
        """
        logger.info("Trying to establish connection with Copernicus API.")
        data = {
            "client_id": "cdse-public",
            "username": username,
            "password": password,
            "grant_type": "password",

            }
        try:
            r = requests.post("https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
            )
            dt_now = datetime.now()
            r.raise_for_status()
        except Exception as e:
            raise Exception(
                f"Keycloak token creation failed. Response from the server was: {r.json()}"
                )

        logger.success("Connection estalished")
        access_token = r.json()["access_token"]
        refresh_token = r.json()["refresh_token"]
        return access_token, refresh_token, dt_now

    def refresh_access_token(self, refresh_token: str):
        """
        Refresh the access token using the provided refresh token.

        Parameters:
        -----------
        refresh_token : str
            The refresh token obtained during the initial authentication process.

        Returns:
        --------
        r.json()["access_token"], dt_now: Tuple[str, datetime]
        A tuple containing the new access token and the current datetime when the refresh was performed.
    """
        data = {
            "client_id": "cdse-public",
            "refresh_token": refresh_token,
            "grant_type": "refresh_token",
        }

        try:
            r = requests.post(
                "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
                data=data,
            )
            dt_now = datetime.now()
            r.raise_for_status()
        except Exception as e:
            raise Exception(
                f"Access token refresh failed. Reponse from the server was: {r.json()}"
            )

        return r.json()["access_token"], dt_now



    def retrieve_sent_prod_from_query(self, params):
        """
        Retrieve Sentinel products from a specified query.

        Parameters:
        -----------
        params: str
            The query parameters to be appended to the URL.

        Returns:
        ---------
        response.json()['value']: list
            List of Sentinel products, each represented as a dictionary.
        None
            if no products are found.
        """
        logger.info("Searching Sentinel products...")
        url = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products"
        url_complet  = url + params
        response = requests.get(url_complet)
        if response.status_code == 200:
            if not response.json()['value']:
                logger.warning("There is no Sentinel data available with this parameters.")
                return None

            for product in response.json()['value']:
                if product['Online']:
                    product_name = response.json()['value']
                    logger.success(f"Found successfully {len(product_name)} products!")
                    return response.json()['value']
        else:
            # If status code is not 200, raise an exception to trigger the retry mechanism
            response.raise_for_status()


    def retrieve_bands_for_resolution(self, session, headers, response, resolution, band_list, url_index):
        """
        Retrieve links for bands at a specific resolution for Sentinel 2 type L2A.

        Parameters:
        -----------
        session: requests.Session
            The active session for making HTTP requests.
        headers: dict
            The headers to include in the HTTP request.
        response: requests.Response
            The response object containing information about the product.
        resolution: str
            The resolution category ('10m', '20m', '60m').
        band_list: list
            List of bands to retrieve links for.
        url_index: int
            The index in the response JSON where the URL for bands is located.

        Returns:
        --------
        url_bands_resolution: list
            List of lists containing band names and their corresponding URLs.

        Example:
        --------
        session = requests.Session()
        headers = {'Authorization': 'Bearer YOUR_ACCESS_TOKEN'}
        response = session.get("https://example.com/api/products/1234")
        resolution = "10m"
        band_list = ["band1", "band2"]
        url_index = 0

        bands_links = retrieve_bands_for_resolution(session, headers, response, resolution, band_list, url_index)

        """
        url_resolution = response.json()["result"][url_index]["Nodes"]["uri"]
        response_resolution = session.get(url_resolution, headers=headers, stream=True)
        url_bands_resolution = []

        for band in band_list:
            url_bands = response_resolution.json()["result"]
            for product in url_bands:
                if band in product["Name"]:
                    item_to_append = [product["Name"], product["Nodes"]["uri"]]
                    url_bands_resolution.append(item_to_append)

        logger.success(f"Links successfully retrieved for bands {band_list} in resolution {resolution}")
        return url_bands_resolution




    def get_links_l1c_product(self, access_token, product_id, bands):
        """
        Retrieve links for bands in a Level-1C (L1C) product.

        Parameters:
        -----------
        access_token: str
            The access token for authentication.
        product_id: str
            The ID of the Sentinel product.
        bands: list
            List of bands to retrieve links for.

        Returns:
        --------
        url_l1c_bands: list
            List of lists containing band names and their corresponding URLs.

        Example:
        --------
        access_token = "YOUR_ACCESS_TOKEN"
        product_id = "1234567890"
        bands = ["band1", "band2"]

        l1c_bands_links = get_links_l1c_product(access_token, product_id, bands)
        """
        url_l1c_bands = []
        headers = {'Authorization': f'Bearer {access_token}'}
        url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/Nodes"

        try:
            logger.info("Starting connection to search for band links...")
            session = requests.Session()
            session.headers.update(headers)
            response = session.get(url, headers=headers, stream=True)

            #### Trying to download specific bands #########################
            url_1 = response.json()['result'][0]["Nodes"]['uri']
            response_1 = session.get(url_1, headers=headers, stream=True)
            url_2 = response_1.json()['result']
            granule_index = next((index for index, d in enumerate(url_2) if d.get('Id') == 'GRANULE'), None)
            if granule_index is not None:
                url_2 = url_2[granule_index]["Nodes"]["uri"]
            response_2 = session.get(url_2, headers=headers, stream=True)
            url_3 = response_2.json()["result"][0]["Nodes"]["uri"]
            response_3 = session.get(url_3, headers=headers, stream=True)
            url_4 = response_3.json()["result"][1]["Nodes"]["uri"]
            response_4 = session.get(url_4, headers=headers, stream=True)
            products = response_4.json()["result"]
            for product in products:
                for band in bands:
                    if band in product["Name"]:
                        itens_to_append = [product["Name"], product["Nodes"]["uri"]]
                        url_l1c_bands.append(itens_to_append)
                        break
            logger.success(f"Links for bands {bands} successfully retrieved")

            return url_l1c_bands

        except Exception as e:
            raise requests.exceptions.RequestException(e)




    def get_links_l2a_product(self, access_token, product_id, bands):
        """
        Retrieve links for bands in a Level-2A (L2A) product.

        Parameters:
        -----------
        access_token: str
            The access token for authentication.
        product_id: str
            The ID of the Sentinel product.
        bands: dict
            Dictionary mapping resolutions to lists of bands.

        Returns:
        --------
        resolution_links: dict
            Dictionary containing resolution-wise lists of band names and their corresponding URLs.

        Example:
        --------
        access_token = "YOUR_ACCESS_TOKEN"
        product_id = "1234567890"
        bands = {"10m": ["band1", "band2"], "20m": ["band3", "band4"]}

        l2a_links = get_links_l2a_product(access_token, product_id, bands)
        """
        resolution_links = {}
        headers = {'Authorization': f'Bearer {access_token}'}
        url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/Nodes"

        try:
            logger.info("Starting connection to search for band links...")
            session = requests.Session()
            session.headers.update(headers)
            response = session.get(url, headers=headers, stream=True)

            #### Trying to download specific bands #########################
            url_1 = response.json()['result'][0]["Nodes"]['uri']
            response_1 = session.get(url_1, headers=headers, stream=True)
            url_2 = response_1.json()['result'][1]["Nodes"]["uri"]
            response_2 = session.get(url_2, headers=headers, stream=True)
            url_3 = response_2.json()["result"][0]["Nodes"]["uri"]
            response_3 = session.get(url_3, headers=headers, stream=True)
            url_4 = response_3.json()["result"][1]["Nodes"]["uri"]
            response_4 = session.get(url_4, headers=headers, stream=True)

            # Mapping resolution to url_index
            resolution_to_url_index = {'10m': 0,
                                        '20m': 1,
                                        '60m': 2
                                        }

            for resolution, band_list in bands.items():
                logger.info(f"Searching links for resolution {resolution}...")
                url_index = resolution_to_url_index.get(resolution, -1)

                if url_index != -1:
                    resolution_links[resolution] = self.retrieve_bands_for_resolution(
                        session, headers, response_4, resolution, band_list, url_index)
                else:
                    # Handle the case when resolution is not found in the mapping
                    logger.warning(f"Resolution {resolution} not mapped to a valid url_index.")


        except Exception as e:
            raise requests.exceptions.RequestException(e)

        return resolution_links



    def retrieve_bands_links(self, access_token, products_info, bands_dict):
        """
        Retrieve links for bands in Sentinel products.

        Parameters:
        -----------
        access_token: str
            The access token for authentication.
        products_info: list
            List of lists containing information about Sentinel products.
        bands_dict: dict
            Dictionary mapping product types to dictionaries of resolutions and their corresponding bands.

        Returns:
        --------
        all_links: dict
            Dictionary containing links for bands in Sentinel products.
            The structure is {product_name: {product_type: {resolution: [(band_name, band_link), ...], ...}, ...}, ...}.

        Example:
        --------
        access_token = "YOUR_ACCESS_TOKEN"
        products_info = [
            ["product_id_1", "product_name_1", "path_1", "date_1", "tile_1", "platform_1", "L1C"],
            ["product_id_2", "product_name_2", "path_2", "date_2", "tile_2", "platform_2", "L2A"],
            ]
        bands_dict = {
            "L1C": ["band1", "band2"],
            "L2A": {"10m": ["band3", "band4"], "20m": ["band5", "band6"]},
            }

        links = retrieve_bands_links(access_token, products_info, bands_dict)
        """
        all_links = {}

        for product_info in products_info:
            product_id = product_info[0]
            product_type = product_info[6]
            product_name = product_info[1]
            product_name = product_name.replace(".SAFE", "")
            logger.info(f"Getting bands links for: {product_name}")

            if product_type == "L1C":
                logger.info(f"{product_name} is type {product_type}...")
                l1c_bands = bands_dict["L1C"]
                links_l1c = self.get_links_l1c_product(access_token, product_id, l1c_bands)
                all_links.setdefault(product_name, {}).setdefault("L1C", links_l1c)
            elif product_type == "L2A":
                logger.info(f"{product_name} is type {product_type}...")
                l2a_bands = bands_dict["L2A"]
                links_l2a = self.get_links_l2a_product(access_token, product_id, l2a_bands)
                all_links.setdefault(product_name, {}).setdefault("L2A", links_l2a)

        return all_links



    def download(self, access_token, url, output_path, product_name, name):
        """
        Download data from a specified URL using an access token.

        Parameters:
        -----------
        access_token: str
            The access token obtained from the Sentinel API for authentication.
        url: str
            The URL of the data to be downloaded.
        output_path: str
            The local path where the downloaded data will be saved.
        product_name: str
            The name of the Sentinel product associated with the data.
        name: str
            The name or identifier for the downloaded data.

        Returns:
        --------
        None
        """
        headers = {'Authorization': f'Bearer {access_token}'}

        try:
            logger.info("Starting connection to download data...")
            session = requests.Session()
            session.headers.update(headers)
            response = session.get(url, headers=headers, stream=True)

            if response.status_code == 200:
                logger.success("Connection estabilished!")
                logger.info(f"Starting download of: {name}")
                # Get the file size from the 'Content-Length' header of the response
                total_size = int(response.headers.get('Content-Length', 0))

                # Open a file for writing in binary mode
                with open(output_path, "wb") as file, tqdm(desc="Downloading", total=total_size, unit="B", unit_scale=True, unit_divisor=1024, ) as bar:
                    # Iterate over the content in chunks (chunk_size=8192 bytes)
                    for chunk in response.iter_content(chunk_size=8192):
                        # Check if the chunk is not empty
                        if chunk:
                            # Write the chunk to the file
                            file.write(chunk)
                            # Update the progress bar with the size of the current chunk
                            bar.update(len(chunk))
                logger.success(f"Data {name} successfuly downloaded for product {product_name}")
            else:
                # If the response status code is not 200, raise an exception
                response.raise_for_status()
        except Exception as e:
            raise requests.exceptions.RequestException(e)

    def download_bands(self, access_token, products_info, bands_links, base_dir, dt_access_token, refresh_token, tile):
        """
        Download bands for Sentinel products based on the provided links.

        Parameters:
        -----------
        access_token: str
            The access token obtained from the Sentinel API for authentication.
        products_info: List[List]
            List of lists containing information about Sentinel products.
        bands_links: Dict[str, Any]
            Dictionary containing links to bands for each Sentinel product.
        base_dir: str
            The base directory where the downloaded bands will be saved.

        Returns:
        --------
        None
        """

        utils = Utils()
        files = Files()
        dates = Dates()
        for product_info in products_info:
            product_id = product_info[0]
            product_type = product_info[6]
            product_name = product_info[1]
            product_date = product_info[3]
            product_platform = product_info[5]
            product_tile = product_info[4]
            product_name = product_name.replace(".SAFE", "")
            logger.info(f"Starting download process for product {product_name}")


            if tile == product_tile:
                if product_name in bands_links:
                    if "MSIL1C" in product_name:
                        product = bands_links[product_name]
                        key = next(iter(product))
                        # Access the nested lists
                        products_link_lists = product[key]
                        for nested_list in products_link_lists:
                            name, link = nested_list
                            link_mod = utils.modify_string(link)
                            filepath = files.check_file_exist(base_dir, product_platform, product_date, product_tile, product_type, name)
                            logger.info(f"Starting download process for band {name}")
                            if filepath[0] == True:
                                logger.warning(f"Band {name} Already downloaded")
                            else:
                                dt_now = datetime.now()
                                expired = dates.is_token_expired(dt_access_token, dt_now)

                                if expired:
                                    access_token, dt_access_token = self.refresh_access_token(refresh_token)
                                logger.info("Data has not been downloaded. Starting data downloaded...")
                                self.download(access_token, link_mod, filepath[1], product_name, name)

                    elif "MSIL2A" in product_name:
                        product = bands_links[product_name]
                        key = next(iter(product))
                        # Access the nested lists
                        products_link_lists = product[key]
                        for resolution, bands_links_list in products_link_lists.items():
                            for band in bands_links_list:
                                name = band[0]
                                link = band[1]
                                link_mod = utils.modify_string(link)
                                filepath = files.check_file_exist(base_dir, product_platform, product_date, product_tile, product_type, name, resolution)

                                logger.info(f"Starting download process for band {name}")
                                if filepath[0] == True:
                                    logger.warning(f"Band {name} Already downloaded")
                                else:
                                    logger.info("Data has not been downloaded. Starting data downloaded...")
                                    dt_now = datetime.now()
                                    expired = dates.is_token_expired(dt_access_token, dt_now)

                                    if expired:
                                        access_token, dt_access_token = self.refresh_access_token(refresh_token)
                                    self.download(access_token, link_mod, filepath[1], product_name, name)

            elif tile == None:
               if product_name in bands_links:
                    if "MSIL1C" in product_name:
                        product = bands_links[product_name]
                        key = next(iter(product))
                        # Access the nested lists
                        products_link_lists = product[key]
                        for nested_list in products_link_lists:
                            name, link = nested_list
                            link_mod = utils.modify_string(link)
                            filepath = files.check_file_exist(base_dir, product_platform, product_date, product_tile, product_type, name)
                            logger.info(f"Starting download process for band {name}")
                            if filepath[0] == True:
                                logger.warning(f"Band {name} Already downloaded")
                            else:
                                dt_now = datetime.now()
                                expired = dates.is_token_expired(dt_access_token, dt_now)

                                if expired:
                                    access_token, dt_access_token = self.refresh_access_token(refresh_token)
                                logger.info("Data has not been downloaded. Starting data downloaded...")
                                self.download(access_token, link_mod, filepath[1], product_name, name)

                    elif "MSIL2A" in product_name:
                        product = bands_links[product_name]
                        key = next(iter(product))
                        # Access the nested lists
                        products_link_lists = product[key]
                        for resolution, bands_links_list in products_link_lists.items():
                            for band in bands_links_list:
                                name = band[0]
                                link = band[1]
                                link_mod = utils.modify_string(link)
                                filepath = files.check_file_exist(base_dir, product_platform, product_date, product_tile, product_type, name, resolution)

                                logger.info(f"Starting download process for band {name}")
                                if filepath[0] == True:
                                    logger.warning(f"Band {name} Already downloaded")
                                else:
                                    logger.info("Data has not been downloaded. Starting data downloaded...")
                                    dt_now = datetime.now()
                                    expired = dates.is_token_expired(dt_access_token, dt_now)

                                    if expired:
                                        access_token, dt_access_token = self.refresh_access_token(refresh_token)
                                    self.download(access_token, link_mod, filepath[1], product_name, name)
            else:
                logger.warning("Tile not selected to download")



In [10]:
from loguru import logger

class Sentinel2_Band_Downloader:
    """
    Class responsible for downloading Sentinel-2 individual bands.

    Attributes:
    -----------
    log_path: str
        The path where logs will be saved.
    output_base_path: str
        The base path where downloaded bands will be saved.

    Methods:
    --------
    __init__(self, log_path: str, output_base_path: str):
        Initializes the Sentinel2_Band_Downloader instance.

    connect_to_api(self, username: str, password: str) -> str:
        Connects to the Sentinel API and obtains an access token.

    construct_query(self, footprint: str, start_date: str, end_date: str, cloud_cover_percentage: int,
                    type: str, platform_name: str) -> str:
        Constructs a query for retrieving Sentinel-2 products based on specified parameters.

    products_from_sentinel_2(self, params: str) -> List[Dict[str, Any]]:
        Retrieves Sentinel-2 products based on the provided query parameters.

    get_products_info(self, products: List[Dict[str, Any]]) -> List[List]:
        Retrieves information about Sentinel-2 products.

    output_folder(self, products_info: List[List], bands_dict: Dict[str, Any]) -> List[str]:
        Creates output folders to save downloaded bands.

    get_bands_links(self, access_token: str, products_info: List[List], bands_dict: Dict[str, Any]) -> Dict[str, Any]:
        Retrieves links to bands for Sentinel-2 products.

    download_band(self, access_token: str, products_info: List[List], bands_link: Dict[str, Any], base_dir: str):
        Downloads bands for Sentinel-2 products based on the provided links.

    download_sentinel2_bands(self, access_token: str, params: str, bands_dict: Dict[str, Any]):
        Orchestrates the download process for Sentinel-2 bands.

    """

    def __init__(self, output_base_path):
        """Initializes the Sentinel2_Band_Downloader instance."""
        self.output_base_path = output_base_path


    def connect_to_api(self, username, password):
        """Connects to the Sentinel API and obtains an access token."""
        connections = Connections()
        access_token, refresh_token, dt_access_token = connections.access_token(username, password)
        return access_token, refresh_token, dt_access_token

    def construct_query(self, footprint, start_date, end_date, cloud_cover_percentage, type, platform_name):
        """Constructs a query for retrieving Sentinel-2 products based on specified parameters."""
        utils = Utils()
        query = utils.construct_query_for_sentinel2_products(footprint, start_date, end_date, cloud_cover_percentage, type, platform_name)
        return query

    def products_from_sentinel_2(self, params):
        """Retrieves Sentinel-2 products based on the provided query parameters."""
        connections = Connections()
        products = connections.retrieve_sent_prod_from_query(params)
        return products

    def get_products_info(self, products):
        """Retrieves information about Sentinel-2 products."""
        utils = Utils()
        products_info = utils.retrieve_products_info(products)
        return products_info

    def output_folder(self, products_info, bands_dict):
        """Creates output folders to save downloaded bands."""
        files = Files()
        directory_paths = files.create_output_folders(self.output_base_path, products_info, bands_dict)
        return directory_paths

    def get_bands_links(self, access_token, products_info, bands_dict):
        """Retrieves links to bands for Sentinel-2 products."""
        connections = Connections()
        bands_links = connections.retrieve_bands_links(access_token, products_info, bands_dict)
        return bands_links

    def download_band(self, access_token, products_info, bands_link, base_dir, dt_access_token, refresh_token, tile):
        """Downloads bands for Sentinel-2 products based on the provided links."""
        connections = Connections()
        connections.download_bands(access_token, products_info, bands_link, base_dir, dt_access_token, refresh_token, tile)


    def download_sentinel2_bands(self, access_token, params, bands_dict, dt_access_token, refresh_token, tile):
        """Orchestrates the download process for Sentinel-2 bands."""
        products = self.products_from_sentinel_2(params)
        if products is None:
            logger.warning("Stopping further execution.")
        else:
            products_info = self.get_products_info(products)
            self.output_folder(products_info, bands_dict)
            links = self.get_bands_links(access_token, products_info, bands_dict)
            self.download_band(access_token, products_info, links, self.output_base_path, dt_access_token, refresh_token, tile)

            return products_info


In [13]:
from itertools import product

def main(shp_path):
    utils = Utils()

    for file in os.listdir(shp_path):
            if file.endswith('.shp'):
                shapefile_path = os.path.join(shp_path, file)
                roi_name = os.path.splitext(file)[0]

                # Specify the path where you want the log file to be created and the data saved
                output_base_path = os.path.join("arquivos/download_sentinel", roi_name)

                #Create an instance of Sentinel2_Band_Downloader
                sentinel_downloader = Sentinel2_Band_Downloader(output_base_path)

                # Call the connect_to_api method with your credentials
                username = "username"
                password = "password"
                access_token, refresh_token, dt_access_token = sentinel_downloader.connect_to_api(username, password)

                # Writing all filter necessary to search a product of Sentinel-2
                footprint = utils.shapefile_to_polygon(shapefile_path)
                start_date = "2023-08-20"
                end_date = "2023-08-31"
                cloud_cover = "10"
                type_str = "L2A"
                type_list = ["L1", "L2A"]
                platform_name = "SENTINEL-2"

                # filter_str = downloader.construct_query(footprint, start_date, end_date, cloud_cover, type_str, platform_name)
                filter_list = sentinel_downloader.construct_query(f'{footprint}', start_date, end_date, cloud_cover, type_str, platform_name)


                # Create a dict with the bands to download
                bands_dict = {"L2A":{"10m": ["B02", "B03", "B04", "B08"]}}

                # Download the data
                sentinel_downloader.download_sentinel2_bands(access_token, filter_list, bands_dict, dt_access_token, refresh_token, None)


if __name__ == "__main__":
    path = 'arquivos/roi'
    main(path)

2024-12-08 18:15:35.235 | INFO     | __main__:access_token:28 - Trying to establish connection with Copernicus API.
2024-12-08 18:15:36.133 | SUCCESS  | __main__:access_token:47 - Connection estalished
2024-12-08 18:15:36.136 | INFO     | __main__:construct_query_for_sentinel2_products:34 - Creating query to download Sentinel data...
2024-12-08 18:15:36.137 | SUCCESS  | __main__:construct_query_for_sentinel2_products:41 - Query created successfully
2024-12-08 18:15:36.137 | INFO     | __main__:retrieve_sent_prod_from_query:104 - Searching Sentinel products...
2024-12-08 18:15:41.708 | SUCCESS  | __main__:retrieve_sent_prod_from_query:116 - Found successfully 1 products!
2024-12-08 18:15:41.709 | INFO     | __main__:retrieve_products_info:71 - Retrieving information about 1 products...
2024-12-08 18:15:41.710 | INFO     | __main__:retrieve_products_info:74 - Retrieving information about product 1
2024-12-08 18:15:41.710 | SUCCESS  | __main__:retrieve_products_info:87 - ID, Name, S3Path,

In [1]:
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from loguru import logger

class Reprojector:
    """
    Class responsable for reproject raster.
    """

    def __init__(self, target_crs="EPSG:4326", compression="lzw"):
        """
        Inicialize reprojection.

        Parameters:
        ------------
        targer_crs: str
          Target CRS to reproject.
        compression: str
          Compression to use.
        """
        self.target_crs = target_crs
        self.compression = compression

    def reproject_file(self, input_file, output_file):
        """
        Reproject raster file to new CRS.

        Parameters:
        ------------
        input_file: str
          Path to input file.
        output_file: str
          Path to output file.
        """

        logger.info(f"Iniciating reprojection for {input_file}")
        with rasterio.open(input_file) as src:
            transform, width, height = calculate_default_transform(
                src.crs, self.target_crs, src.width, src.height, *src.bounds
            )
            kwargs = src.meta.copy()
            kwargs.update({
                "driver": "GTiff",
                "crs": self.target_crs,
                "transform": transform,
                "width": width,
                "height": height,
                "compress": self.compression})  # Apply compression

            with rasterio.open(output_file, 'w', **kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=self.target_crs,
                        resampling=Resampling.bilinear
                    )

In [2]:
import os
from loguru import logger

class SentinelReprojector:
    """
    Class responsible for traversing directories and reproject Sentinel-2 files.
    """

    def __init__(self, base_path, output_path, reprojector):
        """
        Inicialize Sentinel-2 tiles reprojection.

        Parameters:
        -----------
        base_path : str
            Base path to directories to Sentinel-2 files.
        output_path : str
            Path to save reproject images.
        reprojector : Reprojector
            Instacy to Reprojector class.
        """
        self.base_path = base_path
        self.output_path = output_path
        self.reprojector = reprojector

    def run(self):
        """
        Inicialize reprojection.
        """
        for root, dirs, files in os.walk(self.base_path):
            for file in files:
                if file.endswith('.jp2'):
                    input_file = os.path.join(root, file)

                    relative_path = os.path.relpath(root, self.base_path)
                    output_dir = os.path.join(self.output_path, relative_path)
                    os.makedirs(output_dir, exist_ok=True)

                    output_file = os.path.join(output_dir, file)
                    output_file = os.path.splitext(output_file)[0] + ".tif"

                    if os.path.exists(output_file):
                      logger.info(f"File alredady exists, skipping: {output_file}")
                      continue

                    try:
                      self.reprojector.reproject_file(input_file, output_file)
                      logger.success(f"Reprojected: {input_file} -> {output_file}")
                    except Exception as e:
                      logger.error(f"Error to reproject {input_file}: {e}")

In [3]:
def process_sentinel_tiles(base_path, output_path, target_crs="EPSG:4326", compression="lzw"):

    reprojector_instance = Reprojector(target_crs=target_crs, compression=compression)
    sentinel_reprojector = SentinelReprojector(base_path, output_path, reprojector_instance)
    sentinel_reprojector.run()

if __name__ == "__main__":
    base_path = "arquivos/download_sentinel"
    output_path = "arquivos/reproject"

    process_sentinel_tiles(base_path, output_path)

2024-12-04 18:04:05.547 | INFO     | __main__:reproject_file:37 - Iniciating reprojection for arquivos/download_sentinel/7_02_aoi/Sentinel-2/2024/08/11/T22JFN/L2A/10m/T22JFN_20240811T132239_B08_10m.jp2
2024-12-04 18:04:20.920 | SUCCESS  | __main__:run:48 - Reprojected: arquivos/download_sentinel/7_02_aoi/Sentinel-2/2024/08/11/T22JFN/L2A/10m/T22JFN_20240811T132239_B08_10m.jp2 -> arquivos/reproject/7_02_aoi/Sentinel-2/2024/08/11/T22JFN/L2A/10m/T22JFN_20240811T132239_B08_10m.tif
2024-12-04 18:04:20.921 | INFO     | __main__:reproject_file:37 - Iniciating reprojection for arquivos/download_sentinel/7_02_aoi/Sentinel-2/2024/08/11/T22JFN/L2A/10m/T22JFN_20240811T132239_B03_10m.jp2
2024-12-04 18:04:36.841 | SUCCESS  | __main__:run:48 - Reprojected: arquivos/download_sentinel/7_02_aoi/Sentinel-2/2024/08/11/T22JFN/L2A/10m/T22JFN_20240811T132239_B03_10m.jp2 -> arquivos/reproject/7_02_aoi/Sentinel-2/2024/08/11/T22JFN/L2A/10m/T22JFN_20240811T132239_B03_10m.tif
2024-12-04 18:04:36.843 | INFO     | _

In [20]:
import os
from typing import List

class FileHandler:
    def __init__(self, base_path, stage):
        """
        Initializes the file manager for a directory.

        Parameters:
        ------------
        base_path: str 
            Path where cropped file are located
        """
        self.base_path = base_path
        self.stage = stage

    def process_directory(self, current_path, structure_dict):
        """
        Create a dict of dicts with files as a list of files

        Parameters:
        ------------
        current_path: str
            Path where cropped file are located
        
        structure_dict: dict
            Dict to store structure
        """
        if self.stage == "Merge":
            number_parts = 8
        elif self.stage == "Resample":
            number_parts = 5

        for root, dirs, files in os.walk(current_path):
            for file in files:
                if file.endswith(".tif"):
                    # Obtém a estrutura relativa
                    relative_path = os.path.relpath(root, self.base_path)
                    path_parts = relative_path.split(os.sep)

                    # Certifique-se de que há subpastas esperadas
                    if len(path_parts) != number_parts:  # nome_roi, tile, ano, mes, dia
                        logger.warning(f"Ignorando: {root} (Estrutura inesperada)")
                        continue
                    
                    if self.stage == 'Merge':
                        nome_roi, tile, year, month, day = path_parts[0], path_parts[5], path_parts[2], path_parts[3], path_parts[4]

                    elif self.stage == 'Resample':
                        nome_roi, tile, year, month, day = path_parts
                    
                    # Atualiza o dicionário de estrutura
                    structure_dict.setdefault(nome_roi, {})
                    structure_dict[nome_roi].setdefault(tile, {})
                    structure_dict[nome_roi][tile].setdefault(year, {})
                    structure_dict[nome_roi][tile][year].setdefault(month, {})
                    structure_dict[nome_roi][tile][year][month].setdefault(day, [])
                    structure_dict[nome_roi][tile][year][month][day].append(os.path.join(root, file))

    def list_files_by_structure(self):
        """
        Starts the process of listing files organized by ROI, Tile, Year, Month and Day.

        Returns:
        ---------
        strucure_dict: dict
            Stuctured dict
        """
        structure_dict = {}
        self.process_directory(self.base_path, structure_dict)
        return structure_dict

In [14]:
import rasterio

class BandMerger:
    def __init__(self):
        pass

    def merge(self, band_files, output_file):
        """
        """
        if not band_files:
            raise ValueError("List of bands is empty")


        bands = []
        for band_file in band_files:
            with rasterio.open(band_file) as src:
                bands.append(src.read(1))  


        with rasterio.open(band_files[0]) as src:
            meta = src.meta
            meta.update(count=len(bands))  


        with rasterio.open(output_file, "w", **meta) as dst:
            for i, band in enumerate(bands, start=1):
                dst.write(band, i)

        logger.success(f"Arquivo combinado salvo em: {output_file}")


In [15]:
import os

class MergerManager:
    def __init__(self, base_output_path: str):
        """
        Initializes the directory manager to save merged files.

        Parameters:
        ------------
        base_output_path: str
            Base path to save merged files
        """
        self.base_output_path = base_output_path
        self.mergerclass = BandMerger()

    def merger_bands(self, file_structure):
        """
        Creates the necessary directories based on the structure dictionary and applies the merge method        

        Parameters:
        ------------
        file_structure: dict
            Dict with files to merge
        """

        for nome_roi, tiles in file_structure.items():
            for tile, years in tiles.items():
                for year, months in years.items():
                    for month, days in months.items():
                        for day, files in days.items():
                            dir_path = os.path.join(self.base_output_path, nome_roi, tile, year, month, day)

                            if not os.path.exists(dir_path):
                                os.makedirs(dir_path)

                            output_file_name = f"{nome_roi}_{tile}_{year}_{month}_{day}_merged.tif"
                            output_file_path = os.path.join(dir_path, output_file_name)
                            self.mergerclass.merge(files, output_file_path)


In [16]:
import glob
import os

def merge_bands(cropped_path, merge_path):

    files_merger = FileHandler(cropped_path, 'Merge')
    dict_cropped_filed = files_merger.list_files_by_structure()

    mergermanager = MergerManager(merge_path)
    mergermanager.merger_bands((dict_cropped_filed))
    

if __name__ == "__main__":
    cropped_path = "arquivos/reproject"
    merge_path = "arquivos/merge"

    merge_bands(cropped_path, merge_path)

Ignorando: arquivos/reproject/7_02_aoi/Sentinel-2/2024/08/11/T22JFN/L2A/10m (Estrutura inesperada)
Ignorando: arquivos/reproject/7_02_aoi/Sentinel-2/2024/08/11/T22JFN/L2A/10m (Estrutura inesperada)
Ignorando: arquivos/reproject/7_02_aoi/Sentinel-2/2024/08/11/T22JFN/L2A/10m (Estrutura inesperada)
Ignorando: arquivos/reproject/7_02_aoi/Sentinel-2/2024/08/11/T22JFN/L2A/10m (Estrutura inesperada)
Ignorando: arquivos/reproject/7_02_aoi/Sentinel-2/2024/06/02/T22JFN/L2A/10m (Estrutura inesperada)
Ignorando: arquivos/reproject/7_02_aoi/Sentinel-2/2024/06/02/T22JFN/L2A/10m (Estrutura inesperada)
Ignorando: arquivos/reproject/7_02_aoi/Sentinel-2/2024/06/02/T22JFN/L2A/10m (Estrutura inesperada)
Ignorando: arquivos/reproject/7_02_aoi/Sentinel-2/2024/06/02/T22JFN/L2A/10m (Estrutura inesperada)
Ignorando: arquivos/reproject/7_02_aoi/Sentinel-2/2024/02/03/T22JFN/L2A/10m (Estrutura inesperada)
Ignorando: arquivos/reproject/7_02_aoi/Sentinel-2/2024/02/03/T22JFN/L2A/10m (Estrutura inesperada)
Ignorando:

In [5]:
import rasterio

class RasterResample:
    """Class to execute raster operations"""

    def __init__(self) -> None:
        pass

    def resample_bands_rasterio(self, filepath, output_path, xres, yres):
        logger.info(f"Resampling resolution to approximately {xres}x{yres} degrees using Rasterio's Lanczos...")

        # Open the input raster file using Rasterio
        with rasterio.open(filepath) as dataset:

            # Calculate scale factors for the x and y axes based on desired resolution
            scale_factor_x = dataset.res[0] / xres
            scale_factor_y = dataset.res[1] / yres

            # Copy the dataset profile to keep metadata for the output file
            profile = dataset.profile.copy()

            # Read the data from the dataset, applying the Lanczos resampling method to adjust resolution
            data = dataset.read(
                out_shape=(
                    dataset.count,
                    int(dataset.height * scale_factor_y),
                    int(dataset.width * scale_factor_x),
                ),
                resampling=rasterio.enums.Resampling.lanczos,
            )

            # Adjust the affine transform to account for the new scaling
            transform = dataset.transform * dataset.transform.scale(
                (1 / scale_factor_x), (1 / scale_factor_y)
            )

            # Update the profile with the new dimensions, transform, and file format (GTiff)
            profile.update({
                "height": data.shape[1],
                "width": data.shape[2],
                "transform": transform,
                "driver": "GTiff",
                "count": data.shape[0],
            })

            # Write the resampled data to the specified output path with the updated profile
            with rasterio.open(output_path, "w", **profile) as dst:
                dst.write(data)


In [7]:
from loguru import logger
import math

class Calculator:
    """Class to execute raster operations"""

    def __init__(self) -> None:
        pass

    def meters_to_degree(self, meter):
      degrees = meter / 111320

      return degrees

In [8]:
import os

class ResampleManager:
    def __init__(self, base_output_path: str):
        """
        Initializes the directory manager to save merged files.

        Parameters:
        ------------
        base_output_path: str
            Base path to save merged files
        """
        self.base_output_path = base_output_path
        self.rasterresample = RasterResample()
        self.calc = Calculator()

    def resampler_raster(self, file_structure):
        """
        Creates the necessary directories based on the structure dictionary and applies the merge method

        Parameters:
        ------------
        file_structure: dict
            Dict with files to merge
        """
        resolution = self.calc.meters_to_degree(30)
        for nome_roi, tiles in file_structure.items():
            for tile, years in tiles.items():
                for year, months in years.items():
                    for month, days in months.items():
                        for day, files in days.items():
                            dir_path = os.path.join(self.base_output_path, nome_roi, tile, year, month, day)

                            if not os.path.exists(dir_path):
                                os.makedirs(dir_path)

                            output_file_name = f"{nome_roi}_{tile}_{year}_{month}_{day}_resampled.tif"
                            output_file_path = os.path.join(dir_path, output_file_name)
                            self.rasterresample.resample_bands_rasterio(files[0], output_file_path, resolution, resolution)

In [19]:
import glob
import os

def resample_bands(merge_path, resample_path):

    files_merger = FileHandler(merge_path, 'Resample')
    dict_merged_filed = files_merger.list_files_by_structure()

    resamplermanager = ResampleManager(resample_path)
    resamplermanager.resampler_raster((dict_merged_filed))


if __name__ == "__main__":
    merge_path = "arquivos/merge"
    resample_path = "arquivos/resampled"

    resample_bands(merge_path, resample_path)

2024-12-04 23:19:01.418 | INFO     | __main__:resample_bands_rasterio:10 - Resampling resolution to approximately 0.00026949335249730504x0.00026949335249730504 degrees using Rasterio's Lanczos...
2024-12-04 23:19:04.657 | INFO     | __main__:resample_bands_rasterio:10 - Resampling resolution to approximately 0.00026949335249730504x0.00026949335249730504 degrees using Rasterio's Lanczos...
2024-12-04 23:19:07.572 | INFO     | __main__:resample_bands_rasterio:10 - Resampling resolution to approximately 0.00026949335249730504x0.00026949335249730504 degrees using Rasterio's Lanczos...
2024-12-04 23:19:10.545 | INFO     | __main__:resample_bands_rasterio:10 - Resampling resolution to approximately 0.00026949335249730504x0.00026949335249730504 degrees using Rasterio's Lanczos...
2024-12-04 23:19:13.448 | INFO     | __main__:resample_bands_rasterio:10 - Resampling resolution to approximately 0.00026949335249730504x0.00026949335249730504 degrees using Rasterio's Lanczos...
2024-12-04 23:19:16.

In [17]:
import rasterio
import numpy as np

class RasterNormalizer:
    def __init__(self, norm_range=(0, 1), max_dn=10000):
        """
        Inicializa a classe para normalizar raster.
        
        :param norm_range: Intervalo de normalização (default: [-1, 1]).
        :param max_dn: Valor máximo de DN para normalização (default: 10000).
        """
        self.norm_range = norm_range
        self.max_dn = max_dn


    def normalize(self, input_file, output_file):
        """
        Normaliza uma imagem raster para o intervalo especificado.
        
        :param input_file: Caminho do arquivo de entrada (raster multibanda).
        :param output_file: Caminho para salvar o arquivo normalizado.
        """
        with rasterio.open(input_file) as src:
            bands = src.read()  # Lê todas as bandas (shape: [n_bands, height, width])
            profile = src.profile

        # Aplicar normalização
        bands_normalized = self._apply_normalization(bands)

        # Atualizar o perfil para float
        profile.update(dtype='float32')

        # Salvar o arquivo normalizado
        with rasterio.open(output_file, 'w', **profile) as dst:
            for i in range(bands.shape[0]):
                dst.write(bands_normalized[i].astype('float32'), i + 1)

        logger.success(f"Arquivo normalizado salvo em: {output_file}")

    def _apply_normalization(self, bands):
        """
        Aplica a normalização nos dados das bandas.
        
        :param bands: Array NumPy com os valores das bandas.
        :return: Array NumPy com as bandas normalizadas.
        """
        min_val, max_val = self.norm_range
        if self.norm_range == (0, 1):
            return bands / self.max_dn  # Normalizar para [0, 1]
        elif self.norm_range == (-1, 1):
            return 2 * (bands / self.max_dn) - 1  # Normalizar para [-1, 1]
        else:
            raise ValueError("Intervalo de normalização não suportado. Use [0, 1] ou [-1, 1].")


In [18]:
import os

class NormManager:
    def __init__(self, base_output_path: str):
        """
        Initializes the directory manager to save normalized files.

        Parameters:
        ------------
        base_output_path: str
            Base path to save normalizedfiles
        """
        self.base_output_path = base_output_path
        self.normalizer = RasterNormalizer()

    def normalize_raster(self, file_structure, resolution):
        """
        Creates the necessary directories based on the structure dictionary and applies the normalized method

        Parameters:
        ------------
        file_structure: dict
            Dict with files to normalize
        """
        for nome_roi, tiles in file_structure.items():
            for tile, years in tiles.items():
                for year, months in years.items():
                    for month, days in months.items():
                        for day, files in days.items():
                            output_path = os.path.join(self.base_output_path)
                            if not os.path.exists(output_path):
                                os.makedirs(output_path)

                            output_file_name = f"{nome_roi}_{tile}_{year}_{month}_{day}_norm_{resolution}.tif"
                            output_file_path = os.path.join(output_path, output_file_name)
                            self.normalizer.normalize(files[0], output_file_path)

In [52]:
import glob
import os

from cv2 import norm

def normalize_bands(files_path, normalized_path):

    files_merger = FileHandler(files_path, 'Resample')
    dict_norm_filed = files_merger.list_files_by_structure()
    normrmanager = NormManager(normalized_path)
    normrmanager.normalize_raster(dict_norm_filed, '30m')


if __name__ == "__main__":
    resample_path = 'arquivos/resampled'
    normalized_path = 'arquivos/norm/30m'
    merged_path = 'arquivos/merge'

    normalize_bands(resample_path, normalized_path)

arquivos/norm/30m
Arquivo normalizado salvo em: arquivos/norm/30m/7_02_aoi_T22JFN_2024_08_11_norm_30m.tif
arquivos/norm/30m
Arquivo normalizado salvo em: arquivos/norm/30m/7_02_aoi_T22JFN_2024_06_02_norm_30m.tif
arquivos/norm/30m
Arquivo normalizado salvo em: arquivos/norm/30m/7_02_aoi_T22JFN_2024_02_03_norm_30m.tif
arquivos/norm/30m
Arquivo normalizado salvo em: arquivos/norm/30m/7_02_aoi_T22JFN_2024_04_18_norm_30m.tif
arquivos/norm/30m
Arquivo normalizado salvo em: arquivos/norm/30m/7_02_aoi_T22JFN_2024_04_08_norm_30m.tif
arquivos/norm/30m
Arquivo normalizado salvo em: arquivos/norm/30m/7_02_aoi_T22JFN_2024_01_14_norm_30m.tif
arquivos/norm/30m
Arquivo normalizado salvo em: arquivos/norm/30m/7_02_aoi_T22JFN_2024_03_14_norm_30m.tif
arquivos/norm/30m
Arquivo normalizado salvo em: arquivos/norm/30m/7_02_aoi_T22JFN_2024_03_24_norm_30m.tif
arquivos/norm/30m
Arquivo normalizado salvo em: arquivos/norm/30m/7_02_aoi_T22JFN_2024_09_30_norm_30m.tif
arquivos/norm/30m
Arquivo normalizado salvo em

In [96]:
import rasterio
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from loguru import logger

class RasterCropper:
    def __init__(self, common_extent, output_directory):
        """
        Inicializa o RasterCropper com a extensão comum e o diretório de saída.
        
        :param common_extent: BoundingBox com os limites (left, bottom, right, top).
        :param output_directory: Caminho para salvar os arquivos recortados.
        """
        self.common_extent = common_extent
        self.output_directory = output_directory

    def crop(self, image_path, output_filename):
        """
        Recorta a imagem para a extensão comum e salva o resultado.

        :param image_path: Caminho da imagem raster a ser recortada.
        :param output_filename: Nome do arquivo de saída.
        """
        output_path = f"{self.output_directory}/{output_filename}"
        
        with rasterio.open(image_path) as src:
            # Cria um polígono a partir da extensão comum
            common_bounds = box(
                self.common_extent.left,
                self.common_extent.bottom,
                self.common_extent.right,
                self.common_extent.top
            )
            gdf = gpd.GeoDataFrame({"geometry": [common_bounds]}, crs=src.crs)

            # Recorta a imagem para os limites comuns
            out_image, out_transform = mask(src, gdf.geometry, crop=True)
            out_meta = src.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })

            # Salva a nova imagem recortada
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(out_image)
        
        logger.success(f"Imagem recortada salva em: {output_path}")

# Uso da classe RasterCropper
if __name__ == "__main__":
    # Definir a extensão comum
    common_extent = rasterio.coords.BoundingBox(
        left=-49.9740858511183,
        bottom=-29.91467661928992,
        right=-48.82739163624227,
        top=-28.91054438788496
    )

    # Diretório de saída
    output_directory = "arquivos/new_test"

    # Instanciar o RasterCropper
    cropper = RasterCropper(common_extent, output_directory)

    # Recortar imagens
    cropper.crop("arquivos/norm/30m/7_02_aoi_T22JFN_2024_01_14_norm_30m.tif", "cropped_30m.tif")
    cropper.crop("arquivos/norm/10m/7_02_aoi_T22JFN_2024_01_14_norm_10m.tif", "cropped_10m.tif")



In [99]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

class ResolutionAligner:
    def __init__(self, resampling_method=Resampling.bilinear):
        """
        Inicializa o ResolutionAligner com o método de reamostragem.

        :param resampling_method: Método de reamostragem (padrão: Resampling.bilinear).
        """
        self.resampling_method = resampling_method

    def align_resolution(self, source_path, target_path, output_path):
        """
        Alinha a resolução de uma imagem raster à de outra e salva o resultado.

        :param source_path: Caminho da imagem raster de origem.
        :param target_path: Caminho da imagem raster de destino (modelo).
        :param output_path: Caminho para salvar a imagem reprojetada.
        """
        with rasterio.open(source_path) as src:
            with rasterio.open(target_path) as target:
                # Calcula a transformação padrão para ajustar à resolução da imagem alvo
                transform, width, height = calculate_default_transform(
                    src.crs, target.crs, target.width, target.height, *target.bounds
                )

                # Atualiza os metadados da imagem de saída
                kwargs = src.meta.copy()
                kwargs.update({
                    'crs': target.crs,
                    'transform': transform,
                    'width': width,
                    'height': height
                })

                # Reamostra e salva a imagem ajustada
                with rasterio.open(output_path, 'w', **kwargs) as dst:
                    for i in range(1, src.count + 1):
                        reproject(
                            source=rasterio.band(src, i),
                            destination=rasterio.band(dst, i),
                            src_transform=src.transform,
                            src_crs=src.crs,
                            dst_transform=transform,
                            dst_crs=target.crs,
                            resampling=self.resampling_method
                        )
        logger.success(f"Imagem alinhada salva em: {output_path}")


# Uso da classe ResolutionAligner
if __name__ == "__main__":
    # Cria uma instância do alinhador de resolução
    aligner = ResolutionAligner(resampling_method=Resampling.bilinear)

    # Alinha a resolução da imagem de 10m para coincidir com a de 30m
    aligner.align_resolution(
        "arquivos/new_test/cropped_10m.tif",
        "arquivos/new_test/cropped_30m.tif",
        "arquivos/new_test/aligned_10m.tif"
    )


In [103]:
import rasterio
from rasterio.enums import Resampling
import numpy as np

class RasterResampler:
    def __init__(self, target_resolution, resampling_method=Resampling.bilinear):
        """
        Inicializa o RasterResampler com a resolução alvo e o método de reamostragem.

        :param target_resolution: Resolução de pixel desejada (e.g., 0.00026949335249730504 / 3).
        :param resampling_method: Método de reamostragem (padrão: Resampling.bilinear).
        """
        self.target_resolution = target_resolution
        self.resampling_method = resampling_method

    def resample(self, src_path, output_path):
        """
        Reamostra a imagem para uma resolução maior e salva o resultado.

        :param src_path: Caminho da imagem raster de origem.
        :param output_path: Caminho para salvar a imagem reamostrada.
        """
        with rasterio.open(src_path) as src:
            # Calcular o fator de escala para a nova resolução
            scale_factor = src.transform[0] / self.target_resolution
            new_width = int(src.width * scale_factor)
            new_height = int(src.height * scale_factor)

            # Criar uma nova transformação
            transform = src.transform * src.transform.scale(
                src.width / new_width,
                src.height / new_height
            )

            # Atualizar o perfil da imagem de saída
            profile = src.profile
            profile.update({
                'transform': transform,
                'width': new_width,
                'height': new_height
            })

            # Criar o arquivo reamostrado
            with rasterio.open(output_path, 'w', **profile) as dst:
                for i in range(1, src.count + 1):
                    # Criar buffer para os dados reamostrados
                    destination = np.empty((new_height, new_width), dtype=src.dtypes[0])

                    # Reamostrar os dados
                    rasterio.warp.reproject(
                        source=rasterio.band(src, i),
                        destination=destination,
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=src.crs,
                        resampling=self.resampling_method
                    )
                    # Escrever os dados reamostrados
                    dst.write(destination, i)
        
        logger.success(f"Imagem reamostrada salva em: {output_path}")


# Uso da classe RasterResampler
if __name__ == "__main__":
    # Nova resolução para 10m
    new_resolution_10m = 0.00026949335249730504 / 3

    # Instancia o RasterResampler
    resampler = RasterResampler(target_resolution=new_resolution_10m)

    # Reamostra a imagem para a nova resolução
    resampler.resample(
        "arquivos/new_test/aligned_10m.tif",
        "arquivos/new_test/resampled_10m.tif"
    )


In [105]:
import numpy as np
import rasterio

class RasterDimensionAdjuster:
    def __init__(self):
        """
        Inicializa a classe RasterDimensionAdjuster.
        """
        pass

    def adjust_dimensions(self, image_path, target_height, target_width, output_path):
        """
        Ajusta as dimensões de uma imagem raster para as dimensões especificadas.

        :param image_path: Caminho da imagem raster de entrada.
        :param target_height: Altura desejada para a imagem ajustada.
        :param target_width: Largura desejada para a imagem ajustada.
        :param output_path: Caminho para salvar a imagem ajustada.
        """
        with rasterio.open(image_path) as src:
            data = src.read()
            profile = src.profile

            # Criar uma nova matriz com dimensões-alvo
            adjusted_data = np.zeros((data.shape[0], target_height, target_width), dtype=data.dtype)

            # Copiar os valores existentes para a matriz ajustada
            min_height = min(data.shape[1], target_height)
            min_width = min(data.shape[2], target_width)
            adjusted_data[:, :min_height, :min_width] = data[:, :min_height, :min_width]

            # Atualizar o perfil da imagem para as novas dimensões
            profile.update({
                'height': target_height,
                'width': target_width
            })

            # Salvar a imagem ajustada
            with rasterio.open(output_path, 'w', **profile) as dst:
                dst.write(adjusted_data)
        
        logger.sucess(f"Imagem ajustada salva em: {output_path}")


# Uso da classe RasterDimensionAdjuster
if __name__ == "__main__":
    # Instancia o ajustador de dimensões
    adjuster = RasterDimensionAdjuster()

    # Ajusta a imagem para as dimensões corretas
    adjuster.adjust_dimensions(
        "arquivos/new_test/resampled_10m.tif",
        11178,  # Altura desejada
        12765,  # Largura desejada
        "arquivos/new_test/aligned_10m_corrected.tif"
    )


In [106]:
import os
import numpy as np
import rasterio
from rasterio.windows import Window

class PatchCreator:
    def __init__(self, dir_30m, dir_10m, output_base_dir, patch_size_30m, stride_30m):
        """
        Classe para criar patches de imagens de 30m e 10m.

        Args:
            dir_30m (str): Diretório contendo as imagens de 30m.
            dir_10m (str): Diretório contendo as imagens de 10m.
            output_base_dir (str): Diretório base onde os patches serão salvos.
            patch_size_30m (int): Tamanho do patch para 30m (e.g., 32).
            stride_30m (int): Passo da janela deslizante para 30m (e.g., 16).
        """
        self.dir_30m = dir_30m
        self.dir_10m = dir_10m
        self.output_base_dir = output_base_dir
        self.patch_size_30m = patch_size_30m
        self.stride_30m = stride_30m
        self.patch_size_10m = patch_size_30m * 3  # Tamanho ajustado para 10m
        self.stride_10m = stride_30m * 3  # Passo ajustado para 10m
        os.makedirs(output_base_dir, exist_ok=True)

    def create_patches(self):
        """
        Cria patches para todas as imagens no diretório.
        Salva os patches em diretórios separados para 30m e 10m.
        """
        files_30m = sorted(os.listdir(self.dir_30m))
        files_10m = sorted(os.listdir(self.dir_10m))


        
        if len(files_30m) != len(files_10m):
            raise ValueError("O número de arquivos em 30m e 10m não corresponde.")

        for file_30m, file_10m in zip(files_30m, files_10m):
            path_30m = os.path.join(self.dir_30m, file_30m)
            path_10m = os.path.join(self.dir_10m, file_10m)

            output_dir_30m = os.path.join(self.output_base_dir, "patches_30m", file_30m.split('.')[0])
            output_dir_10m = os.path.join(self.output_base_dir, "patches_10m", file_10m.split('.')[0])

            os.makedirs(output_dir_30m, exist_ok=True)
            os.makedirs(output_dir_10m, exist_ok=True)

            logger.info(f"Criando patches para {file_30m} e {file_10m}...")
            self._create_patches_for_image(path_30m, output_dir_30m, "30m", self.patch_size_30m, self.stride_30m)
            self._create_patches_for_image(path_10m, output_dir_10m, "10m", self.patch_size_10m, self.stride_10m)

    def _create_patches_for_image(self, image_path, output_dir, prefix, patch_size, stride):
        """
        Cria patches para uma única imagem.

        Args:
            image_path (str): Caminho para a imagem .tif.
            output_dir (str): Diretório onde os patches serão salvos.
            prefix (str): Prefixo para os nomes dos arquivos (e.g., "30m" ou "10m").
            patch_size (int): Tamanho do patch.
            stride (int): Passo da janela deslizante.
        """
        with rasterio.open(image_path) as src:
            h, w = src.height, src.width
            patch_id = 0

            for y in range(0, h - patch_size + 1, stride):
                for x in range(0, w - patch_size + 1, stride):
                    # Lê o patch
                    window = Window(x, y, patch_size, patch_size)
                    patch = src.read(window=window)  # Lê todas as bandas

                    # Salva o patch como .npy
                    patch_filename = os.path.join(output_dir, f"{prefix}_patch_{patch_id}.npy")
                    np.save(patch_filename, patch)
                    patch_id += 1

            logger.success(f"{patch_id} patches criados e salvos em: {output_dir}")

# Exemplo de uso
if __name__ == "__main__":
    # Diretórios das imagens
    dir_30m = "arquivos/new_test/30m"  # Substitua pelo caminho real para as imagens de 30m
    dir_10m = "arquivos/new_test/10m"  # Substitua pelo caminho real para as imagens de 10m
    output_base_dir = "arquivos/new_test/patches"  # Diretório onde os patches serão salvos

    # Configurações de patch
    patch_size_30m = 32  # Tamanho dos patches para 30m
    stride_30m = 16  # Passo da janela deslizante para 30m

    # Criar os patches
    patch_creator = PatchCreator(dir_30m, dir_10m, output_base_dir, patch_size_30m, stride_30m)
    patch_creator.create_patches()


1
1
Criando patches para cropped_30m.tif e aligned_10m_corrected.tif...
60984 patches criados e salvos em: arquivos/new_test/patches/patches_30m/cropped_30m
60984 patches criados e salvos em: arquivos/new_test/patches/patches_10m/aligned_10m_corrected


In [129]:
import os
import shutil
import random

def split_dataset(base_dir_30m, base_dir_10m, output_dir, train_ratio=0.7, valid_ratio=0.15, test_ratio=0.15):
    """
    Divide subpastas de patches em treino, validação e teste.

    Args:
        base_dir_30m (str): Diretório base contendo as subpastas de patches_30m.
        base_dir_10m (str): Diretório base contendo as subpastas de patches_10m.
        output_dir (str): Diretório base onde serão salvas as pastas train, valid e test.
        train_ratio (float): Proporção de dados para treinamento.
        valid_ratio (float): Proporção de dados para validação.
        test_ratio (float): Proporção de dados para teste.
    """
    # Verificar proporções
    assert abs(train_ratio + valid_ratio + test_ratio - 1.0) < 1e-6, "As proporções devem somar 1.0."

    # Criar diretórios de saída
    for split in ["train", "valid", "test"]:
        for subdir in ["patches_30m", "patches_10m"]:
            os.makedirs(os.path.join(output_dir, split, subdir), exist_ok=True)

    # Listar subpastas de patches
    subfolders_30m = sorted(os.listdir(base_dir_30m))
    subfolders_10m = sorted(os.listdir(base_dir_10m))

    if len(subfolders_30m) != len(subfolders_10m):
        raise ValueError("O número de subpastas em patches_30m e patches_10m não corresponde.")

    # Combinar as subpastas para garantir correspondência
    combined = list(zip(subfolders_30m, subfolders_10m))

    # Embaralhar as subpastas para divisão
    random.seed(42)  # Para garantir reprodução dos resultados
    random.shuffle(combined)

    # Dividir em conjuntos de treino, validação e teste
    num_folders = len(combined)
    train_end = int(num_folders * train_ratio)
    valid_end = train_end + int(num_folders * valid_ratio)

    splits = {
        "train": combined[:train_end],
        "valid": combined[train_end:valid_end],
        "test": combined[valid_end:],
    }

    # Mover as subpastas para os diretórios de saída
    for split, folders in splits.items():
        for folder_30m, folder_10m in folders:
            src_30m = os.path.join(base_dir_30m, folder_30m)
            src_10m = os.path.join(base_dir_10m, folder_10m)

            dest_30m = os.path.join(output_dir, split, "patches_30m", folder_30m)
            dest_10m = os.path.join(output_dir, split, "patches_10m", folder_10m)

            shutil.copytree(src_30m, dest_30m)
            shutil.copytree(src_10m, dest_10m)

    logger.sucess(f"Divisão concluída! Dados salvos em {output_dir}")



In [154]:
import os
import shutil
import random

def split_files(base_dir_30m, base_dir_10m, output_dir, train_ratio=0.7, valid_ratio=0.15, test_ratio=0.15):
    """
    Divide arquivos correspondentes em treino, validação e teste.

    Args:
        base_dir_30m (str): Diretório base contendo os arquivos 30m.
        base_dir_10m (str): Diretório base contendo os arquivos 10m.
        output_dir (str): Diretório onde serão salvos os conjuntos (train, valid, test).
        train_ratio (float): Proporção de dados para treinamento.
        valid_ratio (float): Proporção de dados para validação.
        test_ratio (float): Proporção de dados para teste.
    """
    assert abs(train_ratio + valid_ratio + test_ratio - 1.0) < 1e-6, "As proporções devem somar 1.0."

    # Listar arquivos correspondentes
    files_30m = sorted([f for f in os.listdir(base_dir_30m) if f.endswith(".npy")])
    files_10m = sorted([f for f in os.listdir(base_dir_10m) if f.endswith(".npy")])

    if len(files_30m) != len(files_10m):
        raise ValueError("O número de arquivos em 30m e 10m não corresponde.")

    combined = list(zip(files_30m, files_10m))
    random.seed(42)
    random.shuffle(combined)

    # Dividir arquivos em conjuntos
    num_files = len(combined)
    train_end = int(num_files * train_ratio)
    valid_end = train_end + int(num_files * valid_ratio)

    splits = {
        "train": combined[:train_end],
        "valid": combined[train_end:valid_end],
        "test": combined[valid_end:],
    }

    # Criar diretórios de saída
    for split in splits.keys():
        os.makedirs(os.path.join(output_dir, split, "patches_30m"), exist_ok=True)
        os.makedirs(os.path.join(output_dir, split, "patches_10m"), exist_ok=True)

    # Mover arquivos para os diretórios correspondentes
    for split, files in splits.items():
        for file_30m, file_10m in files:
            src_30m = os.path.join(base_dir_30m, file_30m)
            src_10m = os.path.join(base_dir_10m, file_10m)

            dest_30m_dir = os.path.join(output_dir, split, "patches_30m")
            dest_10m_dir = os.path.join(output_dir, split, "patches_10m")

            # Criar diretórios de destino, caso não existam
            os.makedirs(dest_30m_dir, exist_ok=True)
            os.makedirs(dest_10m_dir, exist_ok=True)

            # Caminho completo para o arquivo de destino
            dest_30m = os.path.join(dest_30m_dir, file_30m)
            dest_10m = os.path.join(dest_10m_dir, file_10m)

            shutil.copy2(src_30m, dest_30m)
            shutil.copy2(src_10m, dest_10m)

    logger.success(f"Divisão concluída! Dados salvos em {output_dir}")

# Caminhos
base_dir_30m = "arquivos/new_test/patches/patches_30m/cropped_30m"
base_dir_10m = "arquivos/new_test/patches/patches_10m/cropped_10m"
output_dir = "arquivos/new_test/dataset2"

# Dividir arquivos
split_files(base_dir_30m, base_dir_10m, output_dir)


Divisão concluída! Dados salvos em arquivos/new_test/dataset2


In [159]:
import os
import torch
from torch.utils.data import Dataset
import numpy as np


class PatchesDataset(Dataset):
    """
    Dataset para carregar patches de 30m e 10m diretamente a partir de dois diretórios contendo arquivos `.npy`.
    """
    def __init__(self, dir_30m, dir_10m):
        """
        Args:
            dir_30m (str): Caminho para os arquivos de 30m (.npy).
            dir_10m (str): Caminho para os arquivos de 10m (.npy).
        """
        # Listar arquivos `.npy` nos dois diretórios
        self.files_30m = sorted([os.path.join(dir_30m, f) for f in os.listdir(dir_30m) if f.endswith(".npy")])
        self.files_10m = sorted([os.path.join(dir_10m, f) for f in os.listdir(dir_10m) if f.endswith(".npy")])

        # Validar correspondência entre os arquivos de 30m e 10m
        if len(self.files_30m) != len(self.files_10m):
            raise ValueError(f"Número de arquivos em {dir_30m} não corresponde a {dir_10m}.")

        # Validar se foram encontrados arquivos
        if not self.files_30m or not self.files_10m:
            raise ValueError("Nenhum arquivo foi encontrado nos diretórios especificados.")
        logger.success(f"Total de arquivos encontrados: {len(self.files_30m)} patches de 30m e {len(self.files_10m)} patches de 10m.")

    def __len__(self):
        return len(self.files_30m)

    def __getitem__(self, idx):
        # Carregar os arquivos `.npy`
        img_30m = np.load(self.files_30m[idx]).astype(np.float32)
        img_10m = np.load(self.files_10m[idx]).astype(np.float32)

        # Converter para tensores
        return torch.from_numpy(img_30m), torch.from_numpy(img_10m)


In [3]:
import torch.nn as nn


class SuperResolutionModel(nn.Module):
    """
    Modelo de super-resolução baseado em uma rede convolucional simples.
    """
    def __init__(self, in_channels=4, out_channels=4):
        super(SuperResolutionModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = self.conv4(x)
        return x


In [8]:
import os
import torch
from torch.utils.data import Dataset
import numpy as np


class PatchesDataset(Dataset):
    """
    Dataset para carregar patches de 30m e 10m diretamente a partir de dois diretórios contendo arquivos `.npy`.
    """
    def __init__(self, dir_30m, dir_10m):
        """
        Args:
            dir_30m (str): Caminho para os arquivos de 30m (.npy).
            dir_10m (str): Caminho para os arquivos de 10m (.npy).
        """
        # Listar arquivos `.npy` nos dois diretórios
        self.files_30m = sorted([os.path.join(dir_30m, f) for f in os.listdir(dir_30m) if f.endswith(".npy")])
        self.files_10m = sorted([os.path.join(dir_10m, f) for f in os.listdir(dir_10m) if f.endswith(".npy")])

        # Validar correspondência entre os arquivos de 30m e 10m
        if len(self.files_30m) != len(self.files_10m):
            raise ValueError(f"Número de arquivos em {dir_30m} não corresponde a {dir_10m}.")

        # Validar se foram encontrados arquivos
        if not self.files_30m or not self.files_10m:
            raise ValueError("Nenhum arquivo foi encontrado nos diretórios especificados.")
        logger.success(f"Total de arquivos encontrados: {len(self.files_30m)} patches de 30m e {len(self.files_10m)} patches de 10m.")

    def __len__(self):
        return len(self.files_30m)

    def __getitem__(self, idx):
        img_30m = np.load(self.files_30m[idx]).astype(np.float32)
        img_10m = np.load(self.files_10m[idx]).astype(np.float32)
    
        # Adicionar dimensões de canal (caso necessário)
        if img_30m.ndim == 2:
            img_30m = np.expand_dims(img_30m, axis=0)
        if img_10m.ndim == 2:
            img_10m = np.expand_dims(img_10m, axis=0)
    
        # Converter para tensores
        return torch.from_numpy(img_30m), torch.from_numpy(img_10m)


In [4]:
class SuperResolutionModel(nn.Module):
    """
    Modelo de super-resolução baseado em uma rede convolucional simples com upsampling inicial.
    """
    def __init__(self, in_channels=4, out_channels=4, scale_factor=3):
        super(SuperResolutionModel, self).__init__()
        self.upsample = nn.Upsample(scale_factor=scale_factor, mode='bilinear', align_corners=False)
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.upsample(x)  # Ajusta a dimensão da entrada para corresponder à saída
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = self.conv4(x)
        return x


In [166]:
from torch.utils.data import DataLoader
import torch
import os


class Trainer:
    """
    Classe para treinar o modelo de super-resolução.
    """
    def __init__(self, train_dir_30m, train_dir_10m, valid_dir_30m, valid_dir_10m, checkpoint_dir, batch_size=16, lr=1e-3, epochs=10, checkpoint_interval=5):
        self.train_dataset = PatchesDataset(train_dir_30m, train_dir_10m)
        self.valid_dataset = PatchesDataset(valid_dir_30m, valid_dir_10m)
        self.train_loader = DataLoader(self.train_dataset, batch_size=batch_size, shuffle=True)
        self.valid_loader = DataLoader(self.valid_dataset, batch_size=batch_size, shuffle=False)

        self.model = SuperResolutionModel()
        self.criterion = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.epochs = epochs
        self.checkpoint_dir = checkpoint_dir
        self.checkpoint_interval = checkpoint_interval

        os.makedirs(checkpoint_dir, exist_ok=True)

    def train(self):
        """
        Treina o modelo.
        """
        best_loss = float('inf')

        for epoch in range(1, self.epochs + 1):
            # Treinamento
            self.model.train()
            train_loss = 0.0
            for img_30m, img_10m in self.train_loader:
                self.optimizer.zero_grad()
                output = self.model(img_30m)
                loss = self.criterion(output, img_10m)
                loss.backward()
                self.optimizer.step()
                train_loss += loss.item()

            train_loss /= len(self.train_loader)

            # Validação
            self.model.eval()
            valid_loss = 0.0
            with torch.no_grad():
                for img_30m, img_10m in self.valid_loader:
                    output = self.model(img_30m)
                    loss = self.criterion(output, img_10m)
                    valid_loss += loss.item()

            valid_loss /= len(self.valid_loader)
            logger.info(f"Epoch {epoch}/{self.epochs}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}")

            # Salvar o melhor modelo
            if valid_loss < best_loss:
                best_loss = valid_loss
                best_model_path = os.path.join(self.checkpoint_dir, "best_model.pth")
                torch.save(self.model.state_dict(), best_model_path)
                logger.success(f"Modelo salvo como o melhor até agora com validação loss: {best_loss:.4f}")

            # Salvar checkpoints regulares
            if epoch % self.checkpoint_interval == 0:
                checkpoint_path = os.path.join(self.checkpoint_dir, f"checkpoint_epoch_{epoch}.pth")
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'train_loss': train_loss,
                    'valid_loss': valid_loss,
                }, checkpoint_path)
                logger.success(f"Checkpoint salvo em: {checkpoint_path}")

    def evaluate(self):
        """
        Avalia o modelo no conjunto de validação.
        """
        best_model_path = os.path.join(self.checkpoint_dir, "best_model.pth")
        self.model.load_state_dict(torch.load(best_model_path))
        self.model.eval()

        valid_loss = 0.0
        with torch.no_grad():
            for img_30m, img_10m in self.valid_loader:
                output = self.model(img_30m)
                loss = self.criterion(output, img_10m)
                valid_loss += loss.item()

        valid_loss /= len(self.valid_loader)
        logger.info(f"Loss no conjunto de validação: {valid_loss:.4f}")


In [167]:
if __name__ == "__main__":
    train_dir_30m = "arquivos/new_test/dataset2/train/patches_30m"
    train_dir_10m = "arquivos/new_test/dataset2/train/patches_10m"
    valid_dir_30m = "arquivos/new_test/dataset2/valid/patches_30m"
    valid_dir_10m = "arquivos/new_test/dataset2/valid/patches_10m"
    checkpoint_dir = "arquivos/new_test/checkpoints"

    trainer = Trainer(
        train_dir_30m, train_dir_10m,
        valid_dir_30m, valid_dir_10m,
        checkpoint_dir,
        batch_size=8, lr=1e-4, epochs=30, checkpoint_interval=5
    )

    trainer.train()
    trainer.evaluate()


Total de arquivos encontrados: 42688 patches de 30m e 42688 patches de 10m.
Total de arquivos encontrados: 9147 patches de 30m e 9147 patches de 10m.
Epoch 1/30, Train Loss: 0.0001, Valid Loss: 0.0000
Modelo salvo como o melhor até agora com validação loss: 0.0000
Epoch 2/30, Train Loss: 0.0000, Valid Loss: 0.0000
Modelo salvo como o melhor até agora com validação loss: 0.0000
Epoch 3/30, Train Loss: 0.0000, Valid Loss: 0.0000
Epoch 4/30, Train Loss: 0.0000, Valid Loss: 0.0000
Modelo salvo como o melhor até agora com validação loss: 0.0000
Epoch 5/30, Train Loss: 0.0000, Valid Loss: 0.0000
Checkpoint salvo em: arquivos/new_test/checkpoints/checkpoint_epoch_5.pth
Epoch 6/30, Train Loss: 0.0000, Valid Loss: 0.0000
Modelo salvo como o melhor até agora com validação loss: 0.0000
Epoch 7/30, Train Loss: 0.0000, Valid Loss: 0.0000
Modelo salvo como o melhor até agora com validação loss: 0.0000
Epoch 8/30, Train Loss: 0.0000, Valid Loss: 0.0000
Modelo salvo como o melhor até agora com validaç

/tmp/ipykernel_3822036/308004227.py:81: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(best_model_path))


Loss no conjunto de validação: 0.0000


In [33]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import rasterio
from rasterio.transform import Affine
from math import ceil

# Definição do modelo
class SuperResolutionModel(nn.Module):
    def __init__(self, in_channels=4, out_channels=4, scale_factor=3):
        super(SuperResolutionModel, self).__init__()
        self.upsample = nn.Upsample(scale_factor=scale_factor, mode='bilinear', align_corners=False)
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.upsample(x)  # Ajusta a dimensão da entrada para corresponder à saída
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = self.conv4(x)
        return x

# Função para normalizar os pixels
def normalize_pixels(image_array):
    min_val = image_array.min()
    max_val = image_array.max()
    return (image_array - min_val) / (max_val - min_val)

# Função para gerar patches
def generate_patches(image_array, patch_size=32, stride=16):
    patches = []
    positions = []
    h, w = image_array.shape[:2]
    for i in range(0, h - patch_size + 1, stride):
        for j in range(0, w - patch_size + 1, stride):
            patch = image_array[i:i+patch_size, j:j+patch_size]
            patches.append(patch)
            positions.append((i, j))
    return np.array(patches), positions

# Dataset customizado para carregar patches
class PatchesDataset(Dataset):
    def __init__(self, patches):
        self.patches = patches

    def __len__(self):
        return len(self.patches)

    def __getitem__(self, idx):
        patch = self.patches[idx]
        patch = np.expand_dims(patch, axis=0)  # Adicionar canal (1 para grayscale)
        patch = np.repeat(patch, 4, axis=0)  # Replicar para 4 canais
        return torch.tensor(patch, dtype=torch.float32)

# Função para reconstruir a imagem
def reconstruct_image(patches, positions, image_shape, patch_size, output_patch_size):
    reconstructed_image = np.zeros(image_shape, dtype=np.float32)
    weight_matrix = np.zeros(image_shape, dtype=np.float32)

    for patch, (i, j) in zip(patches, positions):
        i_scaled = i * (output_patch_size // patch_size)
        j_scaled = j * (output_patch_size // patch_size)
        reconstructed_image[i_scaled:i_scaled+output_patch_size, j_scaled:j_scaled+output_patch_size] += patch
        weight_matrix[i_scaled:i_scaled+output_patch_size, j_scaled:j_scaled+output_patch_size] += 1

    reconstructed_image /= weight_matrix
    return reconstructed_image

# Pipeline completo
def process_and_reconstruct_tif(input_tif_path, model_path, output_image_path, patch_size=32, stride=16, scale_factor=3):
    # Carregar o modelo
    model = SuperResolutionModel(in_channels=4, out_channels=4, scale_factor=scale_factor)
    state_dict = torch.load(model_path)
    model.load_state_dict(state_dict)
    model.eval()

    # Ler e normalizar a imagem
    with rasterio.open(input_tif_path) as src:
        image = src.read(1)  # Lê o primeiro canal (grayscale)
        original_transform = src.transform
        original_resolution = src.res[0]  # Assume que os pixels são quadrados
        original_crs = src.crs
        normalized_image = normalize_pixels(image)

    # Gerar patches e suas posições
    patches, positions = generate_patches(normalized_image, patch_size, stride)

    # Preparar os patches para o DataLoader
    dataset = PatchesDataset(patches)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

    # Processar os patches com o modelo
    processed_patches = []
    with torch.no_grad():
        for batch in dataloader:
            predictions = model(batch)  # Predição do modelo
            processed_patches.append(predictions.numpy())

    # Concatenar todos os patches processados
    processed_patches = np.concatenate(processed_patches, axis=0)
    processed_patches = processed_patches[:, 0, :, :]  # Seleciona apenas o primeiro canal

    # Reconstruir a imagem
    output_patch_size = patch_size * scale_factor
    output_shape = (image.shape[0] * scale_factor, image.shape[1] * scale_factor)
    reconstructed_image = reconstruct_image(processed_patches, positions, output_shape, patch_size, output_patch_size)

    # Calcular o novo transform para a saída
    new_resolution = original_resolution / scale_factor
    new_transform = Affine(original_transform.a / scale_factor, original_transform.b, original_transform.c,
                           original_transform.d, original_transform.e / scale_factor, original_transform.f)

    # Salvar a imagem reconstruída
    with rasterio.open(
        output_image_path,
        'w',
        driver='GTiff',
        height=reconstructed_image.shape[0],
        width=reconstructed_image.shape[1],
        count=1,
        dtype=rasterio.float32,
        crs=original_crs,
        transform=new_transform
    ) as dst:
        dst.write(reconstructed_image, 1)

    print(f"Imagem reconstruída salva em: {output_image_path}")

# Exemplo de uso
input_tif_path = "arquivos/new_test/landsat/LC08_220080_20240314.tif"
model_path = "arquivos/new_test/checkpoints/best_model.pth"
output_image_path = "arquivos/new_test/landsat/output_image.tif"

process_and_reconstruct_tif(input_tif_path, model_path, output_image_path, patch_size=32, stride=16)


/tmp/ipykernel_3825180/701649521.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)


Imagem reconstruída salva em: arquivos/new_test/landsat/output_image.tif


/tmp/ipykernel_3825180/701649521.py:71: RuntimeWarning: invalid value encountered in divide
  reconstructed_image /= weight_matrix


In [4]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import rasterio
from rasterio.transform import Affine
from math import ceil

# Definição do modelo
class SuperResolutionModel(nn.Module):
    def __init__(self, in_channels=4, out_channels=4, scale_factor=3):
        super(SuperResolutionModel, self).__init__()
        self.upsample = nn.Upsample(scale_factor=scale_factor, mode='bilinear', align_corners=False)
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.upsample(x)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = self.conv4(x)
        return x

# Função para normalizar os pixels
def normalize_pixels(image_array):
    min_val = image_array.min()
    max_val = image_array.max()
    return (image_array - min_val) / (max_val - min_val)

# Função para selecionar bandas específicas
def select_bands(dataset, band_indices):
    """
    Seleciona bandas específicas de um dataset.
    Args:
        dataset: Rasterio dataset.
        band_indices: Lista de índices das bandas desejadas (1-indexed).
    Returns:
        Numpy array com as bandas selecionadas (shape: [bands, height, width]).
    """
    bands = []
    for idx in band_indices:
        bands.append(dataset.read(idx))
    return np.stack(bands, axis=0)

# Função para gerar patches
def generate_patches(image_array, patch_size=32, stride=16):
    patches = []
    positions = []
    h, w = image_array.shape[1:]  # Altura e largura para imagens com múltiplos canais
    for i in range(0, h - patch_size + 1, stride):
        for j in range(0, w - patch_size + 1, stride):
            patch = image_array[:, i:i+patch_size, j:j+patch_size]
            patches.append(patch)
            positions.append((i, j))
    return np.array(patches), positions

# Dataset customizado para carregar patches
class PatchesDataset(Dataset):
    def __init__(self, patches):
        self.patches = patches

    def __len__(self):
        return len(self.patches)

    def __getitem__(self, idx):
        patch = self.patches[idx]
        return torch.tensor(patch, dtype=torch.float32)

# Função para reconstruir a imagem
def reconstruct_image(patches, positions, image_shape, patch_size, output_patch_size):
    reconstructed_image = np.zeros(image_shape, dtype=np.float32)
    weight_matrix = np.zeros(image_shape, dtype=np.float32)

    for patch, (i, j) in zip(patches, positions):
        i_scaled = i * (output_patch_size // patch_size)
        j_scaled = j * (output_patch_size // patch_size)
        reconstructed_image[:, i_scaled:i_scaled+output_patch_size, j_scaled:j_scaled+output_patch_size] += patch
        weight_matrix[:, i_scaled:i_scaled+output_patch_size, j_scaled:j_scaled+output_patch_size] += 1

    reconstructed_image /= weight_matrix
    return reconstructed_image

# Pipeline completo
def process_and_reconstruct_tif(input_tif_path, model_path, output_image_path, patch_size=32, stride=16, scale_factor=3):
    # Carregar o modelo
    model = SuperResolutionModel(in_channels=4, out_channels=4, scale_factor=scale_factor)
    state_dict = torch.load(model_path)
    model.load_state_dict(state_dict)
    model.eval()

    # Ler e normalizar a imagem
    with rasterio.open(input_tif_path) as src:
        if src.count > 4:
            # Selecionar bandas específicas
            bands = select_bands(src, [1, 2, 3, 6])  # Bandas 01, 02, 03 e 08 (1-indexed)
        else:
            # Usar todas as bandas se forem exatamente 4
            bands = src.read()

        # Normalizar bandas
        normalized_bands = np.array([normalize_pixels(band) for band in bands])
        original_transform = src.transform
        original_resolution = src.res[0]  # Assume que os pixels são quadrados
        original_crs = src.crs

    # Gerar patches e suas posições
    patches, positions = generate_patches(normalized_bands, patch_size, stride)

    # Preparar os patches para o DataLoader
    dataset = PatchesDataset(patches)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

    # Processar os patches com o modelo
    processed_patches = []
    with torch.no_grad():
        for batch in dataloader:
            predictions = model(batch)
            processed_patches.append(predictions.numpy())

    # Concatenar todos os patches processados
    processed_patches = np.concatenate(processed_patches, axis=0)

    # Reconstruir a imagem
    output_patch_size = patch_size * scale_factor
    output_shape = (bands.shape[0], bands.shape[1] * scale_factor, bands.shape[2] * scale_factor)
    reconstructed_image = reconstruct_image(processed_patches, positions, output_shape, patch_size, output_patch_size)

    # Calcular o novo transform para a saída
    new_resolution = original_resolution / scale_factor
    new_transform = Affine(original_transform.a / scale_factor, original_transform.b, original_transform.c,
                           original_transform.d, original_transform.e / scale_factor, original_transform.f)

    # Salvar a imagem reconstruída
    with rasterio.open(
        output_image_path,
        'w',
        driver='GTiff',
        height=reconstructed_image.shape[1],
        width=reconstructed_image.shape[2],
        count=reconstructed_image.shape[0],
        dtype=rasterio.float32,
        crs=original_crs,
        transform=new_transform
    ) as dst:
        dst.write(reconstructed_image)

    print(f"Imagem reconstruída salva em: {output_image_path}")

# Exemplo de uso
input_tif_path = "arquivos/new_test/landsat/5_03_aoi_landsat.tif"
model_path = "arquivos/new_test/checkpoints/best_model.pth"
output_image_path = "arquivos/new_test/landsat/output_image2.tif"

process_and_reconstruct_tif(input_tif_path, model_path, output_image_path, patch_size=32, stride=16, scale_factor=3)


/tmp/ipykernel_3825966/2043536822.py:91: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)


Imagem reconstruída salva em: arquivos/new_test/landsat/output_image2.tif


/tmp/ipykernel_3825966/2043536822.py:84: RuntimeWarning: invalid value encountered in divide
  reconstructed_image /= weight_matrix


In [6]:
!apt-get update

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1192 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]                
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2454 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1225 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3323 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [45.2 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy/main amd64 Pack

In [7]:
!apt-get install zip -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zip
0 upgraded, 1 newly installed, 0 to remove and 9 not upgraded.
Need to get 176 kB of archives.
After this operation, 544 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zip amd64 3.0-12build2 [176 kB]
Fetched 176 kB in 1s (211 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package zip.
(Reading database ... 27144 files and directories currently installed.)
Preparing to unpack .../zip_3.0-12build2_amd64.deb ...
Unpacking zip (3.0-12build2) ...
Setting up zip (3.0-12build2) ...


In [ ]:
!zip -r dataset.zip arquivos/new_test/dataset2

  adding: arquivos/new_test/dataset2/ (stored 0%)
  adding: arquivos/new_test/dataset2/valid/ (stored 0%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/ (stored 0%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_38503.npy (deflated 54%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_1354.npy (deflated 40%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_23961.npy (deflated 55%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_10385.npy (deflated 43%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_39526.npy (deflated 55%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_26214.npy (deflated 54%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_2854.npy (deflated 53%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_35164.npy (deflated 54%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_59200.npy (deflated 54%)
  adding: arquivos

In [ ]:
!zip -r dataset.zip arquivos/new_test/dataset2

  adding: arquivos/new_test/dataset2/ (stored 0%)
  adding: arquivos/new_test/dataset2/valid/ (stored 0%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/ (stored 0%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_38503.npy (deflated 54%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_1354.npy (deflated 40%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_23961.npy (deflated 55%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_10385.npy (deflated 43%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_39526.npy (deflated 55%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_26214.npy (deflated 54%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_2854.npy (deflated 53%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_35164.npy (deflated 54%)
  adding: arquivos/new_test/dataset2/valid/patches_30m/30m_patch_59200.npy (deflated 54%)
  adding: arquivos

In [1]:
!zip -r checkpoint.zip arquivos/new_test/checkpoints

  adding: arquivos/new_test/checkpoints/ (stored 0%)
  adding: arquivos/new_test/checkpoints/checkpoint_epoch_15.pth (deflated 10%)
  adding: arquivos/new_test/checkpoints/checkpoint_epoch_20.pth (deflated 11%)
  adding: arquivos/new_test/checkpoints/checkpoint_epoch_25.pth (deflated 11%)
  adding: arquivos/new_test/checkpoints/checkpoint_epoch_30.pth (deflated 10%)
  adding: arquivos/new_test/checkpoints/best_model.pth (deflated 7%)
  adding: arquivos/new_test/checkpoints/checkpoint_epoch_10.pth (deflated 10%)
  adding: arquivos/new_test/checkpoints/checkpoint_epoch_5.pth (deflated 10%)


In [2]:
from google.colab import files

# Baixar o arquivo .tar.gz
files.download('dataset.zip')

ModuleNotFoundError: No module named 'google.colab'